# Split Main Table

In [1]:
# Importing necessary libraries
import pandas as pd

## Define Variables and Read CSV File
Variables are being used to make the management of the path to the file easier.

### Local Change
You will need to update the value of **project_path** to correspond with the path to where you cloned the project.

#### Pandas and pyarrow
Using the PyArrow backend for pandas offers several benefits:

1. **Improved Performance**: PyArrow's columnar storage format optimizes data access and retrieval, leading to faster processing speeds.
2. **Enhanced Memory Efficiency**: PyArrow is more memory-efficient, especially when handling string data types, compared to the default NumPy backend.
3. **Better Interoperability**: PyArrow facilitates seamless data exchange between different programming languages, reducing the overhead of data conversion.
4. **Extended Data Type Support**: PyArrow supports a wider range of data types and provides better handling of missing data (NA values) across all types.

These advantages make PyArrow a powerful alternative to the traditional NumPy backend in pandas.

In [2]:
# Define the year as a variable
year = 2023

# Define the project path and subdirectories
project_path = 'C:/Users/glenn/ws/PSF-Survey-Data-Playground/'
raw_path = f'data/{year}/raw/'
clean_path = f'data/{year}/cleaned/'

# Load the dataset using the specified path and pyarrow dtype backend
df = pd.read_csv(f'{project_path}{raw_path}{year}_sharing_data_outside.csv', dtype_backend="pyarrow")

## Adding main_id as a Primary Key and Using it as a Foreign Key in Split Tables

To ensure data integrity and establish clear relationships between tables, I am adding main_id as the primary key in the main table. This unique identifier will serve as a reference point for all related records. By splitting the main table into multiple tables, each new table will include main_id as a foreign key. This approach allows for efficient data organization, easy retrieval, and maintains referential integrity across the database.


In [3]:
# Add a primary key column
df.insert(0, 'main_id', range(1, len(df) + 1))

## Splitting the Table

The current table contains over 400 columns, making it difficult to manage and query efficiently. During exploration, I observed a hidden convention in the column names, where each column follows the format `table.column_name`. To improve the table's usability and maintainability, I will split it into multiple tables based on these conventions. Each new table will be created only if it contains at least six columns. This approach will streamline data management and enhance query performance.

To allow for dynamic naming of columns, a dictionary of all the new dataframes will be created. The `main_id` will be included in each dataframe to maintain referential integrity. The original dataframe `df` will not be modified.

Any column name that does not contain a `.` will be placed in the `main_df`, as well as all column groupings containing five or fewer columns.


In [4]:
# Extract column names by converting to a list
column_names = df.columns.tolist()

# Create a list to store the table names
table_names = []

# Derive table names from column names
for col in column_names:
    if '.' in col:
        table_name = col.split('.')[0]
    else:
        table_name = 'main'
    table_names.append(table_name)

# Create a DataFrame with column names and table names
table_col_xref_df = pd.DataFrame({
    'Column Name': column_names,
    'Table Name': table_names
})

In [5]:
# Get count of each unique table name
unique_table_counts = table_col_xref_df['Table Name'].value_counts()

In [6]:
# Iterate through each unique table name and its count
for table_name, count in unique_table_counts.items():
    if count < 6 and table_name != 'main':

        # Reassign the table name to 'main' for these columns
        table_col_xref_df.loc[table_col_xref_df['Table Name'] == table_name, 'Table Name'] = 'main'

In [7]:
# Update the counts after smaller tables are placed into the main table
# This ensures the variable is updated for completeness and potential future use,
# even though it is no longer used in the current code
unique_table_counts = table_col_xref_df['Table Name'].value_counts()

In [8]:
# Get unique table names
unique_table_names = table_col_xref_df['Table Name'].unique()

# Create a dictionary to store the resulting DataFrames
# Create a dictionary to store the resulting DataFrames
dataframes = {}
for table_name in unique_table_names:
    columns = ['main_id'] + table_col_xref_df[table_col_xref_df['Table Name'] == table_name]['Column Name'].tolist()
    dataframes[f"{table_name}_df"] = df[columns]


Here's a markdown narrative to describe the process:

---

## Updating Column Names and Cleaning DataFrames

To streamline the column names and clean the dataframes, I followed these steps:

1. **Update Column Names**:
    - For each dataframe in the dictionary, except for `main_df`, I updated the column names to only include the values after the dot (`.`). This was done to simplify the column names and make them more readable.
    - The `main_id` column was retained as is.
    - The updated column names were assigned back to the dataframe.

2. **Drop Rows with All NaN Values**:
    - For each dataframe, I dropped rows where all columns, except for `main_id`, contained NaN values. This step ensured that only rows with meaningful data were retained.

3. **Save DataFrames to CSV Files**:
    - Finally, I saved each cleaned dataframe to a CSV file. The files were saved to the specified project path and clean path, with the dataframe name used as the filename.


In [9]:
# Update all the column names to just have the values after the dot, with the except 'main'
for name, dataframe in dataframes.items():
    if name != "main_df":
        new_columns = ['main_id'] + [col.split('.')[-1] if '.' in col else col for col in dataframe.columns[1:]]
        dataframe.columns = new_columns

for name, dataframe in dataframes.items():
    # Drop rows where all columns, except 'main_id', are NaN
    dataframes[name] = dataframe.dropna(subset=dataframe.columns.difference(['main_id']), how='all')

# Save the resulting DataFrames to CSV files
for name, dataframe in dataframes.items():
    dataframe.to_csv(f'{project_path}{clean_path}{name}.csv', index=False)